In [5]:
import pdfplumber
import pandas as pd
import re
import os
from datetime import datetime

# File paths - try different path configurations
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Try different path configurations
possible_paths = [
    "factoring_analysis/data/statements/20240131-statements-0778-.pdf",
    "../factoring_analysis/data/statements/20240131-statements-0778-.pdf", 
    "/Users/nicolachiara/VSCODE/PROJECTS/Onboarding/python/onboardingpython01/factoring_analysis/data/statements/20240131-statements-0778-.pdf"
]

pdf_path = None
for path in possible_paths:
    if os.path.exists(path):
        pdf_path = path
        print(f"Found PDF at: {path}")
        break

if pdf_path is None:
    print("Could not find the PDF file. Please check the file path.")
    raise FileNotFoundError("PDF file not found")

# Set output CSV paths relative to the PDF location
pdf_dir = os.path.dirname(pdf_path)
deposits_csv = os.path.join(pdf_dir, "deposits.csv")
withdrawals_csv = os.path.join(pdf_dir, "withdrawals.csv")

# Extract year from filename
filename = os.path.basename(pdf_path)
year = filename[:4]  # First 4 digits: "2024"

print(f"Processing file: {filename}")
print(f"Extracted year: {year}")

def clean_amount(amount_str):
    """Clean and convert amount string to float"""
    try:
        cleaned = re.sub(r'[^\d.-]', '', str(amount_str).replace(',', ''))
        if cleaned:
            return round(float(cleaned), 2)
        return 0.0
    except:
        return 0.0

def extract_all_transactions(full_text, transaction_type):
    """Extract ALL transactions using a comprehensive approach"""
    transactions = []
    
    # Define section markers based on transaction type
    if transaction_type == "deposits":
        # Look for all deposit-related sections
        section_patterns = [
            ("DEPOSITS AND ADDITIONS", "ELECTRONIC WITHDRAWALS"),
            ("DEPOSITS AND ADDITIONS", "Total Deposits and Additions"),
            ("DEPOSITS AND ADDITIONS (continued)", "ELECTRONIC WITHDRAWALS"),
        ]
        print("🔍 Searching for DEPOSIT transactions...")
    else:  # withdrawals
        section_patterns = [
            ("ELECTRONIC WITHDRAWALS", "FEES"),
            ("ELECTRONIC WITHDRAWALS", "Total Electronic Withdrawals"),
            ("ELECTRONIC WITHDRAWALS (continued)", "FEES"),
        ]
        print("🔍 Searching for WITHDRAWAL transactions...")
    
    # Extract all relevant text sections
    all_sections_text = ""
    
    for start_marker, end_marker in section_patterns:
        start_pos = full_text.find(start_marker)
        if start_pos != -1:
            end_pos = full_text.find(end_marker, start_pos)
            if end_pos == -1:
                end_pos = len(full_text)
            
            section_text = full_text[start_pos:end_pos]
            all_sections_text += section_text + "\n"
            print(f"✅ Found section: '{start_marker}' to '{end_marker}'")
    
    if not all_sections_text:
        print(f"❌ No sections found for {transaction_type}")
        return transactions
    
    # Split into lines and process
    lines = [line.strip() for line in all_sections_text.split('\n') if line.strip()]
    
    # Process each line looking for date patterns
    i = 0
    transaction_count = 0
    
    while i < len(lines):
        line = lines[i]
        
        # Skip obvious header/footer lines
        skip_patterns = [
            'DEPOSITS AND ADDITIONS', 'ELECTRONIC WITHDRAWALS', 'DATE', 
            'DESCRIPTION', 'AMOUNT', 'ACCOUNT NUMBER', 'CHASE PERFORMANCE',
            'PAGE', 'DECEMBER 30', 'TOTAL DEPOSITS', 'TOTAL ELECTRONIC',
            'Chase Bank', 'JPMorgan', 'Columbus, OH', 'Customer Service',
            'www.Chase.com', 'continued', 'Page', 'of'
        ]
        
        if any(skip.upper() in line.upper() for skip in skip_patterns):
            i += 1
            continue
        
        # Look for date pattern at start of line: MM/DD
        date_match = re.match(r'^(\d{2}/\d{2})\s+(.+)', line)
        
        if date_match:
            transaction_count += 1
            date_part = date_match.group(1)
            transaction_text = date_match.group(2)
            
            print(f"\n🎯 Transaction #{transaction_count}: Found date {date_part}")
            print(f"   Initial text: {transaction_text[:100]}...")
            
            # Collect all lines for this transaction
            j = i + 1
            collected_lines = [line]
            
            # Keep collecting lines until we hit another date or section boundary
            while j < len(lines):
                next_line = lines[j].strip()
                
                # Stop conditions
                if re.match(r'^\d{2}/\d{2}\s+', next_line):  # Next transaction
                    break
                if any(stop in next_line.upper() for stop in [
                    'TOTAL DEPOSITS', 'TOTAL ELECTRONIC', 'CHECKING SUMMARY',
                    'FEES', 'SERVICE CHARGES', 'DAILY ENDING BALANCE'
                ]):
                    break
                
                collected_lines.append(next_line)
                transaction_text += ' ' + next_line
                j += 1
            
            print(f"   Collected {len(collected_lines)} lines")
            print(f"   Full text: {transaction_text[:200]}...")
            
            # Extract amount using multiple strategies
            amount = 0.0
            
            # Strategy 1: Look for standalone amount on its own line
            for line_text in collected_lines:
                if re.match(r'^\$?[0-9,]+\.?\d*\s*$', line_text.strip()):
                    amount = clean_amount(line_text)
                    print(f"   💰 Found standalone amount: ${amount}")
                    break
            
            # Strategy 2: Look for amounts in the full text
            if amount == 0.0:
                amount_patterns = [
                    r'\$([0-9,]+\.?\d*)',  # $1,234.56
                    r'([0-9,]+\.\d{2})(?!\d)',  # 1,234.56 (with decimals)
                    r'([0-9]{1,3}(?:,[0-9]{3})+)(?!\d)',  # 1,234 (with commas)
                ]
                
                for pattern in amount_patterns:
                    matches = re.findall(pattern, transaction_text)
                    if matches:
                        amounts = [clean_amount(match) for match in matches]
                        amounts = [amt for amt in amounts if amt > 0]
                        if amounts:
                            amount = max(amounts)  # Take the largest
                            print(f"   💰 Found amount in text: ${amount}")
                            break
                    if amount > 0:
                        break
            
            # Strategy 3: Look for numbers at the end of lines
            if amount == 0.0:
                for line_text in reversed(collected_lines):
                    numbers = re.findall(r'([0-9,]+\.?\d*)', line_text)
                    if numbers:
                        potential_amounts = [clean_amount(num) for num in numbers]
                        potential_amounts = [amt for amt in potential_amounts if amt > 10]  # Filter small numbers
                        if potential_amounts:
                            amount = max(potential_amounts)
                            print(f"   💰 Found amount at line end: ${amount}")
                            break
            
            if amount > 0:
                # Clean description
                description = transaction_text
                # Remove amounts from description
                description = re.sub(r'\$?[0-9,]+\.?\d*', '', description)
                description = ' '.join(description.split())  # Clean whitespace
                
                transaction = {
                    'date': f"{date_part}/{year}",
                    'description': description[:250],  # Limit length
                    'amount': amount
                }
                
                transactions.append(transaction)
                print(f"   ✅ SAVED: {transaction['date']} | ${transaction['amount']} | {transaction['description'][:60]}...")
            else:
                print(f"   ❌ SKIPPED: No valid amount found")
            
            i = j  # Move to next transaction
        else:
            i += 1
    
    print(f"\n📊 Total {transaction_type} found: {len(transactions)}")
    return transactions

# Read and parse the PDF
print("📖 Reading PDF...")
with pdfplumber.open(pdf_path) as pdf:
    full_text = ""
    for page_num, page in enumerate(pdf.pages, 1):
        page_text = page.extract_text()
        if page_text:
            print(f"✅ Extracted text from page {page_num}")
            full_text += page_text + "\n"

print(f"📄 Total text length: {len(full_text)} characters")

# Extract transactions
print("\n" + "="*80)
print("🏦 EXTRACTING ALL DEPOSITS")
print("="*80)
deposits = extract_all_transactions(full_text, "deposits")

print("\n" + "="*80)
print("💸 EXTRACTING ALL WITHDRAWALS")
print("="*80)
withdrawals = extract_all_transactions(full_text, "withdrawals")

# Create DataFrames
deposits_df = pd.DataFrame(deposits)
withdrawals_df = pd.DataFrame(withdrawals)

# Save to CSV files
os.makedirs(os.path.dirname(deposits_csv), exist_ok=True)

if not deposits_df.empty:
    deposits_df.to_csv(deposits_csv, index=False)
    print(f"\n💾 Deposits saved to: {deposits_csv}")
    print(f"📈 Total deposits: ${deposits_df['amount'].sum():,.2f}")
else:
    print("❌ No deposits found")

if not withdrawals_df.empty:
    withdrawals_df.to_csv(withdrawals_csv, index=False)
    print(f"\n💾 Withdrawals saved to: {withdrawals_csv}")
    print(f"📉 Total withdrawals: ${withdrawals_df['amount'].sum():,.2f}")
else:
    print("❌ No withdrawals found")

# Final validation
print(f"\n" + "="*80)
print("🔍 VALIDATION REPORT")
print("="*80)
print("Expected from bank statement:")
print("• Deposits: 23 transactions = $80,163.98")
print("• Withdrawals: 14 transactions = $69,675.57")
print()
print("Our results:")

if not deposits_df.empty:
    parsed_deposits = deposits_df['amount'].sum()
    print(f"• Deposits: {len(deposits)} transactions = ${parsed_deposits:,.2f}")
    diff_deposits = abs(80163.98 - parsed_deposits)
    accuracy = (1 - diff_deposits/80163.98) * 100
    print(f"• Deposits accuracy: {accuracy:.1f}% (difference: ${diff_deposits:,.2f})")
    
if not withdrawals_df.empty:
    parsed_withdrawals = withdrawals_df['amount'].sum()
    print(f"• Withdrawals: {len(withdrawals)} transactions = ${parsed_withdrawals:,.2f}")
    diff_withdrawals = abs(69675.57 - parsed_withdrawals)
    accuracy = (1 - diff_withdrawals/69675.57) * 100
    print(f"• Withdrawals accuracy: {accuracy:.1f}% (difference: ${diff_withdrawals:,.2f})")

# Show sample transactions
if not deposits_df.empty:
    print(f"\n📋 Sample deposits:")
    print(deposits_df.head().to_string(index=False))

if not withdrawals_df.empty:
    print(f"\n📋 Sample withdrawals:")
    print(withdrawals_df.head().to_string(index=False))

print(f"\n🎉 Processing complete!")

Current working directory: /Users/nicolachiara/VSCODE/PROJECTS/Onboarding/python/onboardingpython01/factoring_analysis/notebooks
Found PDF at: /Users/nicolachiara/VSCODE/PROJECTS/Onboarding/python/onboardingpython01/factoring_analysis/data/statements/20240131-statements-0778-.pdf
Processing file: 20240131-statements-0778-.pdf
Extracted year: 2024
📖 Reading PDF...
✅ Extracted text from page 1
✅ Extracted text from page 2
✅ Extracted text from page 3
✅ Extracted text from page 4
✅ Extracted text from page 5
✅ Extracted text from page 6
📄 Total text length: 12744 characters

🏦 EXTRACTING ALL DEPOSITS
🔍 Searching for DEPOSIT transactions...
✅ Found section: 'DEPOSITS AND ADDITIONS' to 'ELECTRONIC WITHDRAWALS'
✅ Found section: 'DEPOSITS AND ADDITIONS' to 'Total Deposits and Additions'

🎯 Transaction #1: Found date 01/04
   Initial text: Fedwire Credit Via: Amerant Bank, N.A./067010509 B/O: Dimassi, C.A. Venezuela Ref: 986.00...
   Collected 4 lines
   Full text: Fedwire Credit Via: Amerant 